In [1]:
# !git clone -b retry_rgb_norm 'https://github.com/Lorenz92/SKADC1.git'
# % cd SKADC1
# !echo $PWD
# !pip install tensorflow==2.4.1 keras==2.4.3

In [2]:
import pandas as pd
import numpy as np

import src.dataset as dataset
import src.config as config 
from src.utils import *
import src.models as models
import src.losses as loss

path = config.TRAIN_PATCHES_FOLDER
%load_ext autoreload
%autoreload 2

np.random.seed(config.RANDOM_SEED)

In [3]:
if 'google.colab' in str(get_ipython()):
  use_colab = True
  print('Running on CoLab')
else:
  use_colab = False
  print('Not running on CoLab')

Not running on CoLab


In [4]:
source_dir = './data/training/patches'

if use_colab:
    # Read file from Colab Notebook
    from google.colab import drive
    drive.mount('/content/drive')
    config.MODEL_WEIGHTS = "/content/drive/My Drive/Colab Notebooks/SKADC1"
    config.IMAGE_PATH = "/content/drive/My Drive/Colab Notebooks/SKADC1/asset/560Mhz_1000h.fits"
    config.TRAIN_DATA_FOLDER = "/content/drive/My Drive/Colab Notebooks/SKADC1/asset"
    source_dir = "/content/drive/My Drive/Colab Notebooks/SKADC1/asset/patches"

In [5]:
# Choose the feature extraction model
# backbone='baseline_44'
backbone='baseline_16'
# backbone='vgg16'

if backbone=='baseline_16':
    # config.patch_dim = 50
    config.patch_dim = 20
    config.resizePatch = True
    config.rpn_stride = 4
    config.num_rois = 16
    # anchors in the original image size
    config.anchor_box_scales = [4, 8, 16, 24, 32, 64]
    # config.anchor_box_scales = [6, 8, 12, 16, 24, 32, 64]
    config.resizeFinalDim = 100
    input_shape_1 = config.resizeFinalDim
elif backbone=='baseline_44':
    config.patch_dim = 20
    config.resizePatch = True
    config.rpn_stride = 8
    config.num_rois = 16
    # config.anchor_box_scales = [16,32,64] # anchors in the original image size
    config.anchor_box_scales = [4, 8, 16, 24, 32, 64]
    config.resizeFinalDim = 100
    input_shape_1 = config.resizeFinalDim
else:
    config.patch_dim = 100
    config.resizePatch = True
    config.rpn_stride = 16
    config.num_rois = 16
    config.resizeFinalDim = 600
    input_shape_1=config.resizeFinalDim
    config.anchor_box_scales = [32,64,128]
    config.in_out_img_size_ratio = config.rpn_stride



config.anchor_num = len(config.anchor_box_ratios)*len(config.anchor_box_scales)
input_shape_2=(None, 4)

print(config.resizePatch)
print(config.rpn_stride)

checkpoint = get_model_last_checkpoint(backbone)
print(f'Model last checkpoint: {checkpoint}')

file_path = f'{config.MODEL_WEIGHTS}/{backbone}'
print(f'Writing configuration on txt file: {config.MODEL_WEIGHTS}/config.txt')

if not os.path.exists(file_path):
        os.makedirs(file_path)
        
f = open(f'{file_path}/config.txt',"w+")
f.write(f'backbone = {backbone}\n config.patch_dim = {config.patch_dim}\n config.resizePatch = {config.resizePatch}\n config.rpn_stride = {config.rpn_stride}\n config.num_rois = {config.num_rois}\n config.anchor_box_scales = {config.anchor_box_scales}\n config.resizeFinalDim = {config.resizeFinalDim}\n input_shape_1 = {input_shape_1}')
f.close()

True
4
Checking model checkpoints in directory /Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/model/baseline_16
Model last checkpoint: None
Writing configuration on txt file: /Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/model/config.txt


In [6]:
# Dataset parsing and loading
# use "subset" in config file to load a small portion of data for development/debugging purposes
ska_dataset = dataset.SKADataset(k=3, print_info=False, show_plot=False, use_pb=False)

Dataset shape: (19222, 15)


,ID,RA (core),DEC (core),RA (centroid),DEC (centroid),FLUX,Core frac,BMAJ,BMIN,PA,SIZE,CLASS,SELECTION,x,y
0,24121695,-0.642227,-29.775242,-0.642316,-29.775211,0.000035,0.000000,5.600,5.551,328.554,1,1,1,19704.519,17712.942
1,20167150,-0.390473,-29.867493,-0.390667,-29.868851,0.029555,0.016924,107.110,43.263,191.258,1,1,1,18401.338,17160.919
2,20045120,-0.021411,-29.643486,-0.021359,-29.643561,0.000653,0.012185,5.037,3.525,276.587,1,1,1,16493.600,18506.577
3,33938384,-0.316375,-29.697647,-0.316375,-29.697647,0.000017,0.000000,0.847,0.266,346.641,2,1,1,18020.319,18182.095
4,7527248,-0.336528,-29.627335,-0.336527,-29.627338,0.000054,0.044763,3.343,1.515,279.006,1,1,1,18125.824,18600.680


  0%|          | 0/19222 [00:00<?, ?it/s]

Loading FITS file /Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/data/training/560Mhz_1000h.fits


100%|██████████| 19222/19222 [00:03<00:00, 6048.45it/s]


Initial dataset shape: (19222, 15)
Found 0 boxes with zero area
Rows to be deleted: 0
New dataset shape: (19222, 15)
Extending dataset with new computed columns...
Final cleaned dataset shape: (19222, 26)

Enlarging bboxes...
DONE - Enlarging bboxes...


,ID,RA (core),DEC (core),RA (centroid),DEC (centroid),FLUX,Core frac,BMAJ,BMIN,PA,...,x2,y2,major_semia_px,minor_semia_px,pa_in_rad,width,height,area_orig,area_cropped,class_label
0,24121695,-0.642227,-29.775242,-0.642316,-29.775211,0.000035,0.000000,5.600,5.551,328.554,...,19710.764566,17719.170105,6.264429,6.218255,5.734349,12.503792,12.461712,155.818657,155.818657,1_1
1,20167150,-0.390473,-29.867493,-0.390667,-29.868851,0.029555,0.016924,107.110,43.263,191.258,...,18509.603417,17209.564571,110.037236,44.503323,3.338082,216.538139,97.294352,21067.937901,21067.937901,1_1
2,20045120,-0.021411,-29.643486,-0.021359,-29.643561,0.000653,0.012185,5.037,3.525,276.587,...,16498.009965,18512.295208,5.738106,4.389739,4.827354,8.820318,11.444852,100.947234,100.947234,1_1
3,33938384,-0.316375,-29.697647,-0.316375,-29.697647,0.000017,0.000000,0.847,0.266,346.641,...,18023.322729,18184.661968,3.031418,2.541842,6.050027,6.014593,5.140670,30.919039,30.919039,2_1
4,7527248,-0.336528,-29.627335,-0.336527,-29.627338,0.000054,0.044763,3.343,1.515,279.006,...,18128.789688,18604.884928,4.236891,2.929772,4.869573,5.937378,8.419429,49.989332,49.989332,1_1



----------
Starting training image preprocessing...

Computing max and min pixel value in order to scale image to RGB range
Max pixel value = 0.0065856808796525
Removing negative noise...
Converting to RGB...
Removing positive noise and rescaling to 0-255 interval...

Mean and stdev of the half-gaussian that best fits with noise distribution:
mu=4.561610766868865e-14, stdev=4.5037468510513567e-07
End of training image preprocessing.


In [7]:
ska_dataset.cleaned_train_df[['width', 'height', 'area_orig', 'area_cropped']].describe()

,width,height,area_orig,area_cropped
count,19222.000000,19222.000000,19222.000000,19222.000000
mean,6.282190,6.285123,44.731314,44.731314
std,3.122242,2.910949,169.529219,169.529219
min,4.964839,4.964839,24.649628,24.649628
25%,5.114121,5.116855,26.770928,26.770928
50%,5.388071,5.394034,29.928541,29.928541
75%,6.193903,6.206960,39.145862,39.145862
max,216.538139,97.294352,21067.937901,21067.937901


In [8]:
ska_dataset.cleaned_train_df[['width', 'height', 'area_orig']].quantile([.1,.2,.3,.4,.5,.6,.7,.8,.9,.95,.98,.99,1.])

,width,height,area_orig
0.10,5.021579,5.022318,25.628098
0.20,5.079478,5.083666,26.354817
0.30,5.153434,5.154979,27.221580
0.40,5.250224,5.256348,28.358833
0.50,5.388071,5.394034,29.928541
0.60,5.596149,5.603978,32.164977
0.70,5.931373,5.932616,36.070023
0.80,6.568193,6.603042,43.494231
0.90,8.241342,8.188366,63.754795
0.95,10.577456,10.646052,100.988437


In [9]:
objects_to_ignore=[20167150, 27514971]


In [13]:
show_plot = False
# ska_dataset.generate_patches(limit=10, plot_patches=show_plot, objects_to_ignore=objects_to_ignore, source_dir=None, rgb_norm=True)
ska_dataset.generate_patches(limit=4000, plot_patches=show_plot, objects_to_ignore=objects_to_ignore, source_dir='./data/training/patches', rgb_norm=True)
# ska_dataset.generate_patches(limit=1000, plot_patches=show_plot, objects_to_ignore=objects_to_ignore, source_dir='./data/training/patches', rgb_norm=False)

/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict(


Retrieving patch 4013_16756_16979_20

Retrieving patch 4014_16766_16979_20

Retrieving patch 4015_16776_16979_20

Retrieving patch 4016_16786_16979_20

Retrieving patch 4017_16796_16979_20

Retrieving patch 4018_16806_16979_20

Retrieving patch 4019_16816_16979_20

Retrieving patch 4020_16826_16979_20

Retrieving patch 4022_16846_16979_20

Retrieving patch 4023_16856_16979_20

Retrieving patch 4025_16876_16979_20

Retrieving patch 4026_16886_16979_20

Retrieving patch 4029_16916_16979_20

Retrieving patch 4030_16926_16979_20

Retrieving patch 4031_16936_16979_20

Retrieving patch 4032_16946_16979_20

Retrieving patch 4033_16956_16979_20

Retrieving patch 4037_16996_16979_20

Retrieving patch 4136_16396_16989_20

Retrieving patch 4137_16406_16989_20

Retrieving patch 4138_16416_16989_20

Retrieving patch 4139_16426_16989_20

Retrieving patch 4140_16436_16989_20

Retrieving patch 4144_16476_16989_20

Retrieving patch 4145_16486_16989_20

Retrieving patch 4149_16526_16989_20

Retrieving 

/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict(


Retrieving patch 4176_16796_16989_20

Retrieving patch 4177_16806_16989_20

Retrieving patch 4178_16816_16989_20

Retrieving patch 4179_16826_16989_20

Retrieving patch 4181_16846_16989_20

Retrieving patch 4182_16856_16989_20

Retrieving patch 4184_16876_16989_20

Retrieving patch 4185_16886_16989_20

Retrieving patch 4188_16916_16989_20

Retrieving patch 4189_16926_16989_20

Retrieving patch 4190_16936_16989_20

Retrieving patch 4191_16946_16989_20

Retrieving patch 4192_16956_16989_20

Retrieving patch 4193_16966_16989_20

Retrieving patch 4194_16976_16989_20


/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict(

...

/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict(


Retrieving patch 9722_16606_17339_20

Retrieving patch 9723_16616_17339_20

Retrieving patch 9726_16646_17339_20

Retrieving patch 9727_16656_17339_20

Retrieving patch 9733_16716_17339_20

Retrieving patch 9734_16726_17339_20

Retrieving patch 9737_16756_17339_20

Retrieving patch 9738_16766_17339_20

Retrieving patch 9739_16776_17339_20

Retrieving patch 9744_16826_17339_20

Retrieving patch 9745_16836_17339_20

Retrieving patch 9746_16846_17339_20

Retrieving patch 9747_16856_17339_20

Retrieving patch 9748_16866_17339_20

Retrieving patch 9749_16876_17339_20

Retrieving patch 9750_16886_17339_20

Retrieving patch 9751_16896_17339_20

Retrieving patch 9761_16996_17339_20

Retrieving patch 9858_16376_17349_20

Retrieving patch 9859_16386_17349_20

Retrieving patch 9860_16396_17349_20

Retrieving patch 9861_16406_17349_20

Retrieving patch 9864_16436_17349_20

Retrieving patch 9865_16446_17349_20

Retrieving patch 9866_16456_17349_20

Retrieving patch 9867_16466_17349_20


/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict(


Retrieving patch 9868_16476_17349_20

Retrieving patch 9869_16486_17349_20

Retrieving patch 9870_16496_17349_20

Retrieving patch 9876_16556_17349_20

Retrieving patch 9877_16566_17349_20

Retrieving patch 9878_16576_17349_20

Retrieving patch 9879_16586_17349_20

Retrieving patch 9880_16596_17349_20

Retrieving patch 9881_16606_17349_20

Retrieving patch 9882_16616_17349_20

Retrieving patch 9883_16626_17349_20

Retrieving patch 9884_16636_17349_20

Retrieving patch 9887_16666_17349_20

Retrieving patch 9888_16676_17349_20

Retrieving patch 9896_16756_17349_20

Retrieving patch 9897_16766_17349_20

Retrieving patch 9898_16776_17349_20

Retrieving patch 9903_16826_17349_20

Retrieving patch 9904_16836_17349_20

Retrieving patch 9905_16846_17349_20

Retrieving patch 9906_16856_17349_20

Retrieving patch 9909_16886_17349_20

Retrieving patch 9910_16896_17349_20

Retrieving patch 9913_16926_17349_20

Retrieving patch 9914_16936_17349_20

Retrieving patch 9917_16966_17349_20

Retrieving 

/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')
/Users/lorenzocellini/AI - Universit…/Deep Learning - Andre Asperti/SKA_project/SKADC1/src/dataset.py:519: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  return df[df[filter]==val][cols].set_index(key).T.to_dict('list')


In [ ]:
# ska_dataset.analyze_class_distribution()

In [ ]:
# ska_dataset.split_train_val_stratified(random_state=42, val_portion=0.2)

# print(len(ska_dataset.train_patch_list))
# print(len(ska_dataset.val_patch_list))


In [ ]:
ska_dataset.split_train_val(random_state=42, val_portion=0.2, balanced=False)

## datagen + calc_rpn -> rpn_net -> rpn_to_roi -> calc_iou -> cls_net

# Training

In [ ]:
# print_img(config.TRAIN_PATCHES_FOLDER, '3_16406_16729_20', show_data=True)


In [ ]:
# # Debugging

# train_patch_list = ska_dataset.train_patch_list
# patches_folder_path=config.TRAIN_PATCHES_FOLDER

# train_datagen = prep.get_anchor_gt(patches_folder_path, ['20_16396_16729_20'], backbone, pixel_mean=None)
# image, [y_rpn_cls_true, y_rpn_reg_true], img_data_aug, _, _, patch_id = next(train_datagen)



### Get FRCNN model

In [ ]:
rpn_model, detector_model, total_model = models.get_train_model(input_shape_1=input_shape_1, input_shape_2=input_shape_2, anchor_num=config.anchor_num, pooling_regions=config.pooling_regions, num_rois=config.num_rois, num_classes=len(ska_dataset.class_list)+1, backbone=backbone, use_expander=False)

rpn_model.summary()
detector_model.summary()
total_model.summary()

### Load weights

In [ ]:
ska_dataset.class_list

In [ ]:
models.load_weigths(rpn_model, detector_model, backbone, resume_train=True, checkpoint=checkpoint)
models.compile_models(rpn_model, detector_model, total_model, rpn_losses=[loss.rpn_loss_cls, loss.rpn_loss_regr], detector_losses=[loss.detector_loss_cls, loss.detector_loss_regr], class_list=ska_dataset.class_list)
# models.compile_models(rpn_model, detector_model, total_model, rpn_losses=[loss.rpn_loss_cls, loss.rpn_loss_regr], detector_losses=[loss.categorical_focal_loss([.25, .25, 0.05], 2.), loss.detector_loss_regr], class_list=ska_dataset.class_list)
# models.compile_models(rpn_model, detector_model, total_model, rpn_losses=[loss.rpn_loss_cls, loss.rpn_loss_regr], detector_losses=[loss.categorical_focal_loss([2, 0.5, 2, 2, 0.05], 2.), loss.detector_loss_regr], class_list=ska_dataset.class_list)



In [ ]:
# Specifically checking backbone weights

# total_model.weights[24:25][0][0][0][0]

In [ ]:
# Check that all of the pretrained weights have been loaded.
import numpy as np
for i, j in zip(total_model.weights, rpn_model.weights): 
    # print(i,j)
    assert np.allclose(i,j), 'Weights don\'t match!'

### Train

- errore "Exception: 'a' cannot be empty unless no samples are taken" quando nel sampling ci sono meno di 4 roi
- errore "None type object is not iterable" dovuto al max(IoUs) quando calc_iou torna None, None, None, None
- patch di debug 1550_16376_16779_100
- capire il parametro classifier_regr_std in che modo influenza il training

- provare normalizzazione valori immagini di input
- provare a far passare più roi anzichè 4

In [ ]:
len(ska_dataset.train_patch_list)

In [ ]:
from src.train import *

val_patch_list = None
# train_frcnn(rpn_model, detector_model, total_model, ['2_16396_16729_20'], val_patch_list, ska_dataset.class_list, num_epochs=100, patches_folder_path=config.TRAIN_PATCHES_FOLDER, backbone=backbone, resume_train=True)
train_frcnn(rpn_model, detector_model, total_model, ska_dataset.train_patch_list, ska_dataset.class_list, num_epochs=100, patches_folder_path=config.TRAIN_PATCHES_FOLDER, backbone=backbone, resume_train=False)

# Validation

In [ ]:
rpn_model_eval, detector_model_eval, total_model_eval = models.get_eval_model(input_shape_1=input_shape_1, input_shape_2=input_shape_2, input_shape_fmap=None, anchor_num=config.anchor_num, pooling_regions=config.pooling_regions, num_rois=config.num_rois, num_classes=len(ska_dataset.class_list)+1, backbone=backbone, use_expander=False)

rpn_model_eval.summary()
detector_model_eval.summary()
total_model_eval.summary()

In [ ]:
# Models used for mAP eval
# cp = '95_frcnn_baseline_44.h5'
cp = '0_frcnn_baseline_16.h5'
models.load_weigths(rpn_model_eval, detector_model_eval, backbone, checkpoint=cp)
models.compile_models(rpn_model_eval, detector_model_eval, total_model_eval, rpn_losses=[loss.rpn_loss_cls, loss.rpn_loss_regr], detector_losses=[loss.detector_loss_cls, loss.detector_loss_regr], class_list=ska_dataset.class_list)

In [ ]:
patch_id = ['1319_16846_16809_20']
# patch_id = ska_dataset.train_patch_list
# print(patch_id)
# gt = pd.read_pickle(f'{config.TRAIN_PATCHES_FOLDER}/{patch_id[0]}/{patch_id[0]}.pkl')
# display(gt['class_label'])

preds, mAP, prec, recall = evaluate_model(rpn_model_eval, detector_model_eval, backbone, patch_id, ska_dataset.class_list, metric_threshold=.5, acceptance_treshold=.5)

In [ ]:
preds

In [ ]:
print(config.patch_dim , float(config.resizeFinalDim))

In [ ]:
print_img(config.TRAIN_PATCHES_FOLDER, '5_16501_16729_50')

In [ ]:
anch = pd.read_pickle(f'{config.EVAL_RESULTS}/2_16396_16729_20/2_16396_16729_20.pkl')
display(anch)

In [ ]:
anch['width'] = anch['x2s'] - anch['x1s']
anch['heght'] = anch['y2s'] - anch['y1s']

In [ ]:
anch.describe() 

In [ ]:
print_img(config.TRAIN_PATCHES_FOLDER, '1319_16846_16809_20', config.EVAL_RESULTS)

TODO - 20210508:
- [X] troncamento rumore a 1e-6 tramite half gaussian

TODO - 20210605:
- [X] modificare RPNNet in modo che dia in output anche la backbone - Lorenzo
- [X] scrivere bene training loop: salvare le loss in un df su disco + salvare pesi modello ad ogni giro (Lorenzo - finire di debuggare)

TODO - 20210620:
- [x] implementare mAP in una funzione che prende come parametro un modello o i suoi pesi
- [x] implementare resNet50
- [x] implementare predicted rois - Lorenzo
- [X] implementare plot loss training - Lorenzo
- [X] finire classe datasetv2 - Alice
- [X] check se su colab le performance sono migliori - Lorenzo

TODO - 20210627
- [X] split dataset su combinazioni classi - Alice
- [x] provare campionamento random patch ed osservare le due distribuzioni - Alice

TODO - 20210703
- [x] sistemare salvataggio loss training loop - Lorenzo
- [x] Riscalare immagini tra 0-255 - Alice
- [x] capire se passare tre immagini diverse come input
- [x] usare media vgg16 per zero-centering - Alice

TODO - 20210705
- [x] sistemare nomi funzioni dataset per trasformazione rgb

TODO - 20210711
- [x] rifattorizzare classe dataset spostando nel costruttore i metodi che calcolano i suoi attributi - Lorenzo
- [x] chek valori pixel in input per resnet
- [x] fare funzione per plottare le predictions
- [ ] trainare tutto su colab

TODO - 20210714
- [x] ragionare su come scalare le immagini fra 0 e 1, attualmente hanno tanti valori schiacciati a 0 e il massimo su tutto il train a a 0.4

TODO - 20210717
- [ ] Ablation study: provare a rimuovere stage4 nella resnet - se c'è tempo
- [x] Provare con nostra pixel_mean e con vgg16 pixel_mean -> per il momento abbiamo scartato la prima opzione
- [x] Fare qualche analisi di distribuzione delle classi/dim box del dataset - Alice
- [x] Aggiungere normalizzazione dopo zero centering per resNet50, sulla base del range globale dell'immagine di training
- [x] Provare pulizia dataset originale sulla base del rumore/flusso - Alice
- [ ] Cambiare nomi di tutto - alla fine
- [x] implementare zero-centering su volare medio RGB delle nostre patch
- [x] Funzione che trova l'ultimo checkpoint in colab prima del load_weights - Lorenzo

TODO - 20210801
- [x] Debuggare training baseline 8 e 16 - L
- [x] Finire prove pulizia dataseet noise variando k - A

TODO - 20210904
- [ ] Provare NMS per rimuovere oggetti crowded
- [ ] In evaluate model eliminare le box con una delle due dim a 0 e quelle che escono dalla patch

1.  Summary
    - riassunto progetto
2.  Background
    - SoTA + teoria di base
3.  System Description
    - descrizione dei nostri modelli e dei loro componenti (moduli)
4.  Experimental setup and results
    - dataset pre processing
    - training environment
    - metrics
    - results
5.  Results and error analysis
6.  Discussion

# Plotting

In [ ]:
loss_history = np.load(f"./model/{backbone}/loss_history.npy")
print(loss_history.shape)

In [ ]:
loss_history

In [ ]:
plot_loss(loss_history[1:])

In [ ]:
loss_history[:,1].shape

In [ ]:
np.convolve(loss_history[1], np.ones(10), 'valid') / 10

In [ ]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

lsma_0 = moving_average(loss_history[100:,2], 200)


In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.plot(np.arange(0, len(lsma_0)), lsma_0, 'r')
plt.title('rpn cls')

## Modelli da trainare:
epoch lenght = 200
- baseline_16: patch 50 -> 100, [8,16,32,64]-[1:1], 16roi, limit=4000, 10k epoche, focal_loss, norm: media, no bilanciamento dataset, rgbNorm=True - A
- baseline_16: patch 50 -> 100, [8,16,32,64]-[1:1], 16roi, limit=4000, 10k epoche, NO focal_loss, norm: media, no bilanciamento dataset, rgbNorm=True - A

- baseline_16: patch 20->100, [4,8,16,24,32,64]-[1:1,1:2,2:1], 16roi, limit=4000, 10k epoche, no focal_loss, norm: media, no bilanciamento dataset, rgbNorm=True - L
- baseline_16: patch 20->100, [4,8,16,24,32,64]-[1:1,1:2,2:1], 16roi, limit=4000, 10k epoche, focal_loss, norm: media, no bilanciamento dataset, rgbNorm=True- L


- baseline_44: patch 20->100, [4,8,16,24,32,64]-[1:1,1:2,2:1], 16roi, limit=4000, 10k epoche, no focal_loss, norm: media, no bilanciamento dataset, rgbNorm=True - L
- baseline_44: patch 20->100, [4,8,16,24,32,64]-[1:1,1:2,2:1], 16roi, limit=4000, 10k epoche, focal_loss, norm: media, no bilanciamento dataset, rgbNorm=True - L

- vgg16: 100 -> 600, [32,64,128,256,512]-[1:1,1:2,2:1], 16roi, limit=4000, 10k epoche, no focal_loss, norm: media, no bilanciamento dataset, rgbNorm=True